In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

In [ ]:
#nhl_df=pd.read_csv("nhl.csv")
#cities=pd.read_html("wikipedia_data.html")[1]
#cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nhl_correlation():
    nhl_df=pd.read_csv("assets/nhl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    # YOUR CODE HERE
    nhl_df = nhl_df[nhl_df.year == 2018]
    nhl_df = nhl_df[["team", "W", "L"]]
    def removestar(team):
        if "*" in team:
            return re.findall("([\w\s]*)\*", team)[0]
        else:
            return team
    nhl_df["team"] = nhl_df.team.apply(removestar)
    
    def removebracket(team):
        if "[" in team:
            return re.findall("([\w\s]*)\[", team)[0]
        else:
            return team
    cities["NHL"] = cities["NHL"].apply(removebracket)
    
    def removecity(metro):
        if metro == "New York City":
            return "New York"
        return metro
    cities["Metropolitan area"] = cities["Metropolitan area"].apply(removecity)
    
    #return cities
    cities = cities[["Metropolitan area", "Population (2016 est.)[8]", "NHL"]]
    
    nhl_df["team_only"] = nhl_df.team.apply(lambda x: x.rsplit(None, 1)[-1])

    nhl_df.loc[3, "team_only"] = "Maple Leafs"
    nhl_df.loc[5, "team_only"] = "Red Wings"
    nhl_df.loc[13, "team_only"] = "Blue Jackets"
    nhl_df.loc[27, "team_only"] = "Golden Knights"
    
    nhl_df.loc[14, "team_only"] = "RangersIslandersDevils"
    nhl_df.loc[16, "team_only"] = "RangersIslandersDevils"
    nhl_df.loc[17, "team_only"] = "RangersIslandersDevils"
    
    nhl_df.loc[28, "team_only"] = "KingsDucks"
    nhl_df.loc[30, "team_only"] = "KingsDucks"

    cities.rename(columns = {"Metropolitan area": "city","Population (2016 est.)[8]": "population", "NHL": "team_only" }, inplace = True)
    
    nhl_df = nhl_df[~nhl_df.W.str.contains("Division")]
    
    nhl_df.W = nhl_df.W.astype(float)
    nhl_df.L= nhl_df.L.astype(float)
    #return nhl_df
    nhl_df = nhl_df.groupby(by = "team_only")["W", "L"].mean()
    nhl_df.reset_index(inplace = True)
    #return nhl_df
    #return cities
    merge = pd.merge(nhl_df, cities, how = "inner", on = "team_only")
    merge["W/L%"] = merge.W / (merge.W + merge.L)
    #return merge["W/L%"]
    
    
    #raise NotImplementedError()
    
    population_by_region = pd.to_numeric(merge["population"], errors = "coerce") # pass in metropolitan area population from cities
    win_loss_by_region = pd.to_numeric(merge["W/L%"], errors = "coerce") # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [ ]:
#nba_df=pd.read_csv("nba.csv")
#cities=pd.read_html("wikipedia_data.html")[1]
#cities=cities.iloc[:-1,[0,3,5,6,7,8]]


def nba_correlation():
    nba_df=pd.read_csv("assets/nba.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    # YOUR CODE HERE
    nba_df = nba_df[nba_df.year == 2018]
    nba_df = nba_df[["team", "W", "L"]]
    def removenote(team):
        if "*" in team:
            return re.findall("([\w\s\d]*)\*", team)[0]
        else: 
            return team
    
    nba_df["team"] = nba_df["team"].apply(removenote)
    def removepar(team):
        if "(" in team:
            return re.findall("([\w\s\d]*)\(", team)[0]
        else:
            return team
    
    nba_df["team"] = nba_df["team"].apply(removepar)
    nba_df["team_only"] = nba_df.team.apply(lambda x: x.rsplit(None, 1)[-1])
    
    cities = cities[["Metropolitan area", "Population (2016 est.)[8]", "NBA"]]
    def removebracket(team):
        if "[" in team:
            return re.findall("([\w\s\d]*)\[", team)[0]
        else:
            return team
    cities["NBA"] = cities["NBA"].apply(removebracket)
    nba_df.loc[10, "team_only"] = "KnicksNets"
    nba_df.loc[11, "team_only"] = "KnicksNets"
    nba_df.loc[17, "team_only"] = "Trail Blazers"
    nba_df.loc[24, "team_only"] = "LakersClippers"
    nba_df.loc[25, "team_only"] = "LakersClippers"
    cities.rename(columns = {"Metropolitan area": "city","Population (2016 est.)[8]": "population", "NBA": "team_only" }, inplace = True)
    nba_df.W = nba_df.W.astype(float)
    nba_df.L= nba_df.L.astype(float)
    nba_df = nba_df.groupby(by = "team_only")["W", "L"].mean()
    nba_df.reset_index(inplace = True)
    
    merge2 = pd.merge(nba_df, cities, how = "inner", on = "team_only")
    merge2["W/L%"] = merge2.W / (merge2.W + merge2.L)
    #return merge2
    #return nba_df
    #return cities
    #raise NotImplementedError()
    
    population_by_region = pd.to_numeric(merge2["population"] , errors = "coerce") # pass in metropolitan area population from cities
    win_loss_by_region = pd.to_numeric(merge2["W/L%"], errors = "coerce") # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [ ]:
#mlb_df=pd.read_csv("mlb.csv")
#cities=pd.read_html("wikipedia_data.html")[1]
#cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def mlb_correlation():
    mlb_df=pd.read_csv("assets/mlb.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    # YOUR CODE HERE
    mlb_df = mlb_df[mlb_df.year == 2018]
    mlb_df = mlb_df[["team", "W", "L"]]
    cities = cities[["Metropolitan area", "Population (2016 est.)[8]", "MLB"]]
    def removebracket(team):
        if "[" in team:
            return re.findall("([\w\s\d]*)\[", team)[0]
        else:
            return team
    cities["MLB"] = cities["MLB"].apply(removebracket)
    mlb_df["team_only"] = mlb_df.team.apply(lambda x: x.rsplit(None, 1)[-1])
    
    #return mlb_df
    mlb_df.loc[1, "team_only"] = "YankeesMets"
    mlb_df.loc[18, "team_only"] = "YankeesMets"
    mlb_df.loc[25, "team_only"] = "DodgersAngels"
    mlb_df.loc[13, "team_only"] = "DodgersAngels"
    mlb_df.loc[28, "team_only"] = "GiantsAthletics"
    mlb_df.loc[11, "team_only"] = "GiantsAthletics"
    mlb_df.loc[21, "team_only"] = "CubsWhite Sox"
    mlb_df.loc[8, "team_only"] = "CubsWhite Sox"
    mlb_df.loc[0, "team_only"] = "Red Sox"
    mlb_df.loc[3, "team_only"] = "Blue Jays"
    #return mlb_df
    cities.rename(columns = {"Metropolitan area": "city","Population (2016 est.)[8]": "population", "MLB": "team_only" }, inplace = True)
    mlb_df.W = mlb_df.W.astype(float)
    mlb_df.L= mlb_df.L.astype(float)
    mlb_df = mlb_df.groupby(by = "team_only")["W", "L"].mean()
    mlb_df.reset_index(inplace = True)
    
    merge3 = pd.merge(mlb_df, cities, how = "inner", on = "team_only")
    merge3["W/L%"] = merge3.W / (merge3.W + merge3.L)
    #return cities
    #return mlb_df
    #return merge3
    #raise NotImplementedError()
    
    population_by_region = pd.to_numeric(merge3["population"] , errors = "coerce") # pass in metropolitan area population from cities
    win_loss_by_region = pd.to_numeric(merge3["W/L%"], errors = "coerce") # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [ ]:
#nfl_df=pd.read_csv("nfl.csv")
#cities=pd.read_html("wikipedia_data.html")[1]
#cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nfl_correlation():
    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    # YOUR CODE HERE
    nfl_df = nfl_df[nfl_df.year == 2018]
    nfl_df = nfl_df[["team", "W", "L"]]
    cities = cities[["Metropolitan area", "Population (2016 est.)[8]", "NFL"]]
    def removenote(team):
        if "*" in team:
            return re.findall("([\w\s\d]*)\*", team)[0]
        else: 
            return team
    
    nfl_df["team"] = nfl_df["team"].apply(removenote)
    def removeplus(team):
        if "+" in team:
            return re.findall("([\w\s\d]*)\+", team)[0]
        else:
            return team
    nfl_df["team"] = nfl_df["team"].apply(removeplus)
    
    def removebracket(team):
        if "[" in team:
            return re.findall("([\w\s\d]*)\[", team)[0]
        else:
            return team
    cities["NFL"] = cities["NFL"].apply(removebracket)
    nfl_df["team_only"] = nfl_df.team.apply(lambda x: x.rsplit(None, 1)[-1])
    nfl_df = nfl_df[~nfl_df.W.str.contains("AFC")]
    nfl_df = nfl_df[~nfl_df.W.str.contains("NFC")]
    nfl_df.loc[4, "team_only"] = "GiantsJets"
    nfl_df.loc[24, "team_only"] = "GiantsJets"
    nfl_df.loc[36, "team_only"] = "RamsChargers"
    nfl_df.loc[17, "team_only"] = "RamsChargers"
    nfl_df.loc[38, "team_only"] = "49ersRaiders"
    nfl_df.loc[19, "team_only"] = "49ersRaiders"
    
    cities.rename(columns = {"Metropolitan area": "city","Population (2016 est.)[8]": "population", "NFL": "team_only" }, inplace = True)
    nfl_df.W = nfl_df.W.astype(float)
    nfl_df.L= nfl_df.L.astype(float)
    nfl_df = nfl_df.groupby(by = "team_only")["W", "L"].mean()
    nfl_df.reset_index(inplace = True)
    
    merge4 = pd.merge(nfl_df, cities, how = "inner", on = "team_only")
    merge4["W/L%"] = merge4.W / (merge4.W + merge4.L)
    #return merge4
    #return nfl_df
    #return cities
    
    #raise NotImplementedError()
    
    population_by_region = pd.to_numeric(merge4["population"] , errors = "coerce") # pass in metropolitan area population from cities
    win_loss_by_region = pd.to_numeric(merge4["W/L%"], errors = "coerce") # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [ ]:
#mlb_df=pd.read_csv("mlb.csv")
#nhl_df=pd.read_csv("nhl.csv")
#nba_df=pd.read_csv("nba.csv")
#nfl_df=pd.read_csv("nfl.csv")
#cities=pd.read_html("wikipedia_data.html")[1]
#cities=cities.iloc[:-1,[0,3,5,6,7,8]]


def nhl():
    cities=pd.read_html("wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    nhl_df=pd.read_csv("nhl.csv")
    nhl_df = nhl_df[nhl_df.year == 2018]
    nhl_df = nhl_df[["team", "W", "L"]]
    def removestar(team):
        if "*" in team:
            return re.findall("([\w\s]*)\*", team)[0]
        else:
            return team
    nhl_df["team"] = nhl_df.team.apply(removestar)
    
    def removebracket(team):
        if "[" in team:
            return re.findall("([\w\s]*)\[", team)[0]
        else:
            return team
    cities["NHL"] = cities["NHL"].apply(removebracket)
    
    def removecity(metro):
        if metro == "New York City":
            return "New York"
        return metro
    cities["Metropolitan area"] = cities["Metropolitan area"].apply(removecity)
    
    #return cities
    cities = cities[["Metropolitan area", "Population (2016 est.)[8]", "NHL"]]
    
    nhl_df["team_only"] = nhl_df.team.apply(lambda x: x.rsplit(None, 1)[-1])

    nhl_df.loc[3, "team_only"] = "Maple Leafs"
    nhl_df.loc[5, "team_only"] = "Red Wings"
    nhl_df.loc[13, "team_only"] = "Blue Jackets"
    nhl_df.loc[27, "team_only"] = "Golden Knights"
    
    nhl_df.loc[14, "team_only"] = "RangersIslandersDevils"
    nhl_df.loc[16, "team_only"] = "RangersIslandersDevils"
    nhl_df.loc[17, "team_only"] = "RangersIslandersDevils"
    
    nhl_df.loc[28, "team_only"] = "KingsDucks"
    nhl_df.loc[30, "team_only"] = "KingsDucks"

    cities.rename(columns = {"Metropolitan area": "city","Population (2016 est.)[8]": "population", "NHL": "team_only" }, inplace = True)
    
    nhl_df = nhl_df[~nhl_df.W.str.contains("Division")]
    
    nhl_df.W = nhl_df.W.astype(float)
    nhl_df.L= nhl_df.L.astype(float)
    #return nhl_df
    nhl_df = nhl_df.groupby(by = "team_only")["W", "L"].mean()
    nhl_df.reset_index(inplace = True)
    #return nhl_df
    #return cities
    merge = pd.merge(nhl_df, cities, how = "inner", on = "team_only")
    merge["W/L%"] = merge.W / (merge.W + merge.L)
    #return merge[["city","W/L%"]]
    return merge

def nba():
    cities=pd.read_html("wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    nba_df=pd.read_csv("nba.csv")
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    # YOUR CODE HERE
    nba_df = nba_df[nba_df.year == 2018]
    nba_df = nba_df[["team", "W", "L"]]
    def removenote(team):
        if "*" in team:
            return re.findall("([\w\s\d]*)\*", team)[0]
        else: 
            return team
    
    nba_df["team"] = nba_df["team"].apply(removenote)
    def removepar(team):
        if "(" in team:
            return re.findall("([\w\s\d]*)\(", team)[0]
        else:
            return team
    
    nba_df["team"] = nba_df["team"].apply(removepar)
    nba_df["team_only"] = nba_df.team.apply(lambda x: x.rsplit(None, 1)[-1])
    
    cities = cities[["Metropolitan area", "Population (2016 est.)[8]", "NBA"]]
    def removebracket(team):
        if "[" in team:
            return re.findall("([\w\s\d]*)\[", team)[0]
        else:
            return team
    cities["NBA"] = cities["NBA"].apply(removebracket)
    nba_df.loc[10, "team_only"] = "KnicksNets"
    nba_df.loc[11, "team_only"] = "KnicksNets"
    nba_df.loc[17, "team_only"] = "Trail Blazers"
    nba_df.loc[24, "team_only"] = "LakersClippers"
    nba_df.loc[25, "team_only"] = "LakersClippers"
    cities.rename(columns = {"Metropolitan area": "city","Population (2016 est.)[8]": "population", "NBA": "team_only" }, inplace = True)
    nba_df.W = nba_df.W.astype(float)
    nba_df.L= nba_df.L.astype(float)
    nba_df = nba_df.groupby(by = "team_only")["W", "L"].mean()
    nba_df.reset_index(inplace = True)
    
    merge2 = pd.merge(nba_df, cities, how = "inner", on = "team_only")
    merge2["W/L%"] = merge2.W / (merge2.W + merge2.L)
    #return merge2[["city","W/L%"]]
    return merge2

def mlb():
    cities=pd.read_html("wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    mlb_df=pd.read_csv("mlb.csv")
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    # YOUR CODE HERE
    mlb_df = mlb_df[mlb_df.year == 2018]
    mlb_df = mlb_df[["team", "W", "L"]]
    cities = cities[["Metropolitan area", "Population (2016 est.)[8]", "MLB"]]
    def removebracket(team):
        if "[" in team:
            return re.findall("([\w\s\d]*)\[", team)[0]
        else:
            return team
    cities["MLB"] = cities["MLB"].apply(removebracket)
    mlb_df["team_only"] = mlb_df.team.apply(lambda x: x.rsplit(None, 1)[-1])
    
    #return mlb_df
    mlb_df.loc[1, "team_only"] = "YankeesMets"
    mlb_df.loc[18, "team_only"] = "YankeesMets"
    mlb_df.loc[25, "team_only"] = "DodgersAngels"
    mlb_df.loc[13, "team_only"] = "DodgersAngels"
    mlb_df.loc[28, "team_only"] = "GiantsAthletics"
    mlb_df.loc[11, "team_only"] = "GiantsAthletics"
    mlb_df.loc[21, "team_only"] = "CubsWhite Sox"
    mlb_df.loc[8, "team_only"] = "CubsWhite Sox"
    mlb_df.loc[0, "team_only"] = "Red Sox"
    mlb_df.loc[3, "team_only"] = "Blue Jays"
    #return mlb_df
    cities.rename(columns = {"Metropolitan area": "city","Population (2016 est.)[8]": "population", "MLB": "team_only" }, inplace = True)
    mlb_df.W = mlb_df.W.astype(float)
    mlb_df.L= mlb_df.L.astype(float)
    mlb_df = mlb_df.groupby(by = "team_only")["W", "L"].mean()
    mlb_df.reset_index(inplace = True)
    
    merge3 = pd.merge(mlb_df, cities, how = "inner", on = "team_only")
    merge3["W/L%"] = merge3.W / (merge3.W + merge3.L)
    #return merge3[["city","W/L%"]]
    return merge3

def nfl():
    cities=pd.read_html("wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    nfl_df=pd.read_csv("nfl.csv")
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    # YOUR CODE HERE
    nfl_df = nfl_df[nfl_df.year == 2018]
    nfl_df = nfl_df[["team", "W", "L"]]
    cities = cities[["Metropolitan area", "Population (2016 est.)[8]", "NFL"]]
    def removenote(team):
        if "*" in team:
            return re.findall("([\w\s\d]*)\*", team)[0]
        else: 
            return team
    
    nfl_df["team"] = nfl_df["team"].apply(removenote)
    def removeplus(team):
        if "+" in team:
            return re.findall("([\w\s\d]*)\+", team)[0]
        else:
            return team
    nfl_df["team"] = nfl_df["team"].apply(removeplus)
    
    def removebracket(team):
        if "[" in team:
            return re.findall("([\w\s\d]*)\[", team)[0]
        else:
            return team
    cities["NFL"] = cities["NFL"].apply(removebracket)
    nfl_df["team_only"] = nfl_df.team.apply(lambda x: x.rsplit(None, 1)[-1])
    nfl_df = nfl_df[~nfl_df.W.str.contains("AFC")]
    nfl_df = nfl_df[~nfl_df.W.str.contains("NFC")]
    nfl_df.loc[4, "team_only"] = "GiantsJets"
    nfl_df.loc[24, "team_only"] = "GiantsJets"
    nfl_df.loc[36, "team_only"] = "RamsChargers"
    nfl_df.loc[17, "team_only"] = "RamsChargers"
    nfl_df.loc[38, "team_only"] = "49ersRaiders"
    nfl_df.loc[19, "team_only"] = "49ersRaiders"
    
    cities.rename(columns = {"Metropolitan area": "city","Population (2016 est.)[8]": "population", "NFL": "team_only" }, inplace = True)
    nfl_df.W = nfl_df.W.astype(float)
    nfl_df.L= nfl_df.L.astype(float)
    nfl_df = nfl_df.groupby(by = "team_only")["W", "L"].mean()
    nfl_df.reset_index(inplace = True)
    
    merge4 = pd.merge(nfl_df, cities, how = "inner", on = "team_only")
    merge4["W/L%"] = merge4.W / (merge4.W + merge4.L)
    #return merge4[["city","W/L%"]]
    return merge4

def create_df(sport):
    if sport == "NHL":
        return nhl()
    if sport == "NBA":
        return nba()
    if sport == "MLB":
        return mlb()
    if sport == "NFL":
        return nfl()

def sports_team_performance():
    # YOUR CODE HERE
    #raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    for i in sports:
        for j in sports:
            if i != j:
                merge = pd.merge(create_df(i), create_df(j), how = "inner", on = ["city"])
                p_values.loc[i,j] = stats.ttest_rel(merge["W/L%_x"], merge["W/L%_y"])[1]
    #assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NFL p-value should be around 0.02"
    #assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values